- 这段代码用于访问localhost:6379的redis数据库，检查db0中所有名称含有”class“的hash中所有键值对（即存储的用户名——主页）是否匹配
- 带有一定的矫正策略：
    - 以键为标准删除重复的键值对（重复者以最后一次插入的为准）；
    - 请求每一个值对应的页面，尝试获取response中的用户名：
        - 若用户名完全不匹配，则检查链接对应的键是否存在：
            - 若存在，删除该键值对；
            - 若不存在，将键修改为response中的用户名；
        - 若用户名部分匹配，则将该键修改为response中获取的用户名；


In [67]:
import smallLib as helper
import re # regular expressions
import redis
import selenium.webdriver as webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait #等待页面加载某些元素
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
#暴露了密码。但是这个密码是弱密码，不是重要的密码，所以就不改了
pool = redis.ConnectionPool(host='127.0.0.1',port=6379,db=0,password='123456',decode_responses=True)   
r = redis.Redis(connection_pool=pool)

In [68]:
keys = r.keys()
# 在生产环境中不要使用keys，因为keys会阻塞redis，redis是一个单进程的数据库，所以会阻塞所有的请求
def RedisScan(host,port,password,db=0):
    client = redis.Redis(host=host,port=port,password=password,db=db)
    cursor,counts = 0,0
    while True:
        cursor,keys = client.scan(cursor,match="*",count=100)
        counts += len(keys)
        for key in keys:
            print(key.decode("utf-8"))
        if cursor == 0:
            break
    print("Total Key Number:%s"%counts)
    return  keys

#keys = RedisScan("127.0.0.1", 6379, "123456", 0)

In [69]:
def get_username(url): 
    r = redis.Redis(connection_pool=pool)
    chrome_options =  webdriver.ChromeOptions()
    #这一行很关键，让selenium不加载图片，专心爬
    chrome_options.add_argument('blink-settings=imagesEnabled=false')
    preferences = {
        "webrtc.ip_handling_policy": "disable_non_proxied_udp",
        "webrtc.multiple_routes_enabled": False,
        "webrtc.nonproxied_udp_enabled": False
    }
    chrome_options.add_experimental_option("prefs", preferences)
    #打开url
    s = Service('C:\Program Files\chromedriver\chromedriver.exe')
    driver = webdriver.Chrome(service= s ,options=chrome_options)
    #driver.get("chrome://version/",proxies = proxies)
    driver.get(url=url)
    #此处防止滑动验证码
    try:
        wait=WebDriverWait(driver,3)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME,'captcha-disable-scroll')))
        driver.close()
        return ''
    except:
        pass
    try:
        wait=WebDriverWait(driver,5)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME,'dy-account-close')))
        a = driver.find_elements(by= By.CLASS_NAME,value="dy-account-close")
        time.sleep(1)
        if len(a) >0 :
            a[0].click()
    except:
        selector = 'head > title'
        name = driver.find_elements(by= By.CSS_SELECTOR, value= selector)
        cnt = 0
        while cnt < 5 & len(name)== 0 :
            cnt +=1 
            time.sleep(1)
            name = driver.find_elements(by= By.CSS_SELECTOR, value= selector)
        
        if len(name) == 0:
            print(url)
        else:
            print(name[0].get_attribute('innerHTML'))    
        realname = re.findall(r'(.*)的主页',name[0].get_attribute('innerHTML'))[0]
        driver.close()
        return realname
        
    #关闭登录弹窗
    #获取页面内容
    #hint:先取所有的li标签，再取a标签中的href
    selector = 'head > title'
    name = driver.find_elements(by= By.CSS_SELECTOR, value= selector)
    cnt = 0
    while cnt < 5 & len(name)== 0 :
        cnt +=1 
        time.sleep(1)
        name = driver.find_elements(by= By.CSS_SELECTOR, value= selector)
    
    if len(name) == 0:
        print(url)
    else:
        print(name[0].get_attribute('innerHTML'))    
    realname = re.findall(r'(.*)的主页',name[0].get_attribute('innerHTML'))[0]
    driver.close()
    return realname


In [70]:
dicts = []
for key in keys:
    if key.startswith("class"):
        dicts.append(r.hgetall(key))


In [71]:
replace_dict = {}
for dict in dicts:
    replace_dict = {}
    for key in dict:
        page_name = get_username(dict[key])
        if(len(page_name) > 0):
            if page_name != key:
                replace_dict[key] = page_name
    print(replace_dict)
    for key in replace_dict:
        url = dict.pop(key)
        dict[replace_dict[key]] = url




医心医意海燕的主页 - 承德医学院附属医院妇产科主治医师 王海艳 - 抖音
医路向前巍子的主页 - 北京大学第一医院密云医院急诊外科医师 - 抖音
中医谭医生的主页 - 湘潭市中心医院针灸科副主任医师 谭灿灿 - 抖音
中医内科吴金荣主任的主页 - 北京中医药大学东直门医院内科主任医师 - 抖音
中医黎明的主页 - 山西省中医院中医科主任医师 - 抖音


Exception ignored in: <function Service.__del__ at 0x000001E7AD4BA660>
Traceback (most recent call last):
  File "d:\Anaconda\envs\pygptest\Lib\site-packages\selenium\webdriver\common\service.py", line 183, in __del__
    self.stop()
  File "d:\Anaconda\envs\pygptest\Lib\site-packages\selenium\webdriver\common\service.py", line 149, in stop
    self.send_remote_shutdown_command()
  File "d:\Anaconda\envs\pygptest\Lib\site-packages\selenium\webdriver\common\service.py", line 133, in send_remote_shutdown_command
    if not self.is_connectable():
           ^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\envs\pygptest\Lib\site-packages\selenium\webdriver\common\service.py", line 122, in is_connectable
    return utils.is_connectable(self.port)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\envs\pygptest\Lib\site-packages\selenium\webdriver\common\utils.py", line 102, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
              ^^^^^^^^^^^^^^^^^^^^^^^^^

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00386643]
	(No symbol) [0x0031BE21]
	(No symbol) [0x0021DA9D]
	(No symbol) [0x001FEF6A]
	(No symbol) [0x00273AAB]
	(No symbol) [0x002861B6]
	(No symbol) [0x0026FB76]
	(No symbol) [0x002449C1]
	(No symbol) [0x00245E5D]
	GetHandleVerifier [0x005FA142+2497106]
	GetHandleVerifier [0x006285D3+2686691]
	GetHandleVerifier [0x0062BB9C+2700460]
	GetHandleVerifier [0x00433B10+635936]
	(No symbol) [0x00324A1F]
	(No symbol) [0x0032A418]
	(No symbol) [0x0032A505]
	(No symbol) [0x0033508B]
	BaseThreadInitThunk [0x76A17D69+25]
	RtlInitializeExceptionChain [0x77B5BB9B+107]
	RtlClearBits [0x77B5BB1F+191]


In [ ]:
origin_table = "InformationResearch"
temp_table = "InformationResearch_temp"
r.hmset(origin_table,dicts[1])

C:\Users\Dr.Seydlitz\AppData\Local\Temp\ipykernel_23600\1151907460.py:3: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  r.hmset(origin_table,dicts[1])


True